## 批量转换pdf为xml

In [11]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

import win32com.client, win32com.client.makepy, os, winerror, errno, re
from win32com.client.dynamic import ERRORS_BAD_CONTEXT
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def get_list_files(path):
    ret = []
    for root, _, files in os.walk(path):
        for filespath in files:
            ret.append(os.path.join(root, filespath))
    return ret


def pdf2xml(pdf_file,xml_file):
    src = os.path.abspath(pdf_file)
    ERRORS_BAD_CONTEXT.append(winerror.E_NOTIMPL)
    win32com.client.makepy.GenerateFromTypeLibSpec('Acrobat')
    avDoc = win32com.client.DispatchEx('AcroExch.AVDoc')
    avDoc.Open(src, src)
    pdDoc = avDoc.GetPDDoc()
    jObject = pdDoc.GetJSObject()
    jObject.SaveAs(xml_file, "com.adobe.acrobat.xml-1-00")
    avDoc.Close(-1)
    
def change_all_files_to_xml(path):
    file_list = get_list_files(path)
    path = 'C://Users/USER/Documents/JupyterLab-3/金融产品指导师/NLP/需要分析的说明书xml/'
    for i in tqdm(range(len(file_list))):
        file = file_list[i]
        file_name = file.split('/')[-1][:-4]
        pdf2xml(file,path+file_name+'.xml')

In [3]:
change_all_files_to_xml('./需要分析的说明书pdf/')

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [02:59<00:00,  4.08s/it]


## 提取每个XMl中的所有表格

In [79]:
def transfer_xml_table_to_pandas(xml_table):
    table=[]
    for tr in xml_table.find_all('TR'):
        thl=[]
        tdl=[]
        if tr.find_all('TH') != []:
            for th in tr.find_all('TH'):
                thl.append(th.text.strip().replace('\n',''))
        
        if tr.find_all('TD') != []:
            tdl=[]
            for td in tr.find_all('TD'):
                tdl.append(td.text.strip().replace('\n',''))
        
        if thl==[]:
            table.append(tdl)
        elif tdl==[]:
            table.append(thl)
        else:
            table.append([thl[0],tdl[0]])
                
    df = pd.DataFrame(table)
    return df


def revise_break_off_table(tables):
    for i in range(len(tables)):
        tables[i] = tables[i].reset_index(drop=True)
        table = tables[i]
        for idx in list(table[table[0]==''].index):
            tables[i].iloc[idx-1,1] = tables[i].iloc[idx-1,1]+tables[i].iloc[idx,1]
        tables[i].drop(index=table[table[0]==''].index,inplace=True)
        # 对表1的特殊处理
        if i == 0:
            idxs = list(table[table[0]=='回'].index)
            if idxs != []:
                for idx in idxs:
                    tables[i].loc[idx-1,1] = tables[i].loc[idx-1,1]+tables[i].loc[idx,1]
                    tables[i].loc[idx-1,0] = tables[i].loc[idx-1,0]+tables[i].loc[idx,0]
                    tables[i].drop(index=idx,inplace=True)
        
    return tables


def get_all_tables(xml_path):
    for _, _, files in os.walk(xml_path, topdown=False):
        all_tables={}
        for file in files:
            if file[-4:] == '.xml':
                print(file)
                xml = open(xml_path+file,encoding='utf-8').read()
                xml = xml[:xml.find('<P>投资者账户')]
                soup = BeautifulSoup(xml,'xml')
                dfl = []
                for table in soup.find_all('Table'):
                    df = transfer_xml_table_to_pandas(table)
                    if table.previous_sibling.previous_sibling.name == 'Table' or\
                       table.previous_sibling.previous_sibling.previous_sibling.previous_sibling.name == 'Table':
                        dfl[-1] = dfl[-1].append(df)
                    else:
                        dfl.append(df)
                    
                all_tables[file] = dfl

    clean_tables = {}
    for name,tables in all_tables.items():
        clean_tables[name] = revise_break_off_table(tables)

    return clean_tables

In [80]:
clean_tables = get_all_tables('./需要分析的说明书xml/')

招商银行年年享5号理财计划说明书.xml
招商银行年年享6号理财计划说明书.xml
招商银行美元金葵一年定开1号理财计划产品说明书.xml
招商银行美元金葵一年定开2号理财计划产品说明书.xml
招商银行美元金葵一年定开3号理财计划产品说明书.xml
招商银行美元金葵一年定开4号理财计划产品说明书.xml
招商银行美元金葵一年定开6号理财计划产品说明书.xml
招商银行美元金葵一年定开7号理财计划产品说明书.xml
招商银行美元金葵季季开1号理财计划产品说明书.xml
招商银行美元金葵季季开2号理财计划产品说明书.xml
招商银行美元金葵季季开3号理财计划产品说明书.xml
招商银行美元金葵季季开4号理财计划产品说明书.xml
招商银行美元金葵季季开5号理财计划产品说明书.xml
招商银行美元金葵季季开6号理财计划产品说明书.xml
招商银行零售青葵系列一年半定开1号理财计划产品说明书（产品代码：301037）.xml
招商银行零售青葵系列一年半定开4号理财计划产品说明书（产品代码：301040）.xml
招商银行零售青葵系列一年半定开5号理财计划产品说明书.xml
招商银行零售青葵系列一年定开1号理财计划产品说明书.xml
招商银行零售青葵系列一年定开8号理财计划产品说明书（产品代码：301032）.xml
招商银行零售青葵系列一年定开9号理财计划产品说明书（产品代码：301033）.xml
招商银行零售青葵系列半年定开2号理财计划说明书（产品代码：301002）.xml
招商银行零售青葵系列半年定开3号理财计划说明书（产品代码：301003）.xml
招商银行零售青葵系列半年定开4号理财计划产品说明书.xml
招商银行青葵系列一年半定开005号理财计划产品说明书.xml
招商银行青葵系列一年半定开006号理财计划产品说明书.xml
招商银行青葵系列一年半定开007号理财计划产品说明书（产品代码：301049）.xml
招商银行青葵系列一年半定开008号理财计划产品说明书（产品代码：301050）.xml
招商银行青葵系列一年半定开009号理财计划产品说明书.xml
招商银行青葵系列一年半定开010号理财计划产品说明书（产品代码：301052）.xml
招商银行青葵系列一年半定开011号理财计划产品说明书.xml
招商银行青葵系列一年半定开012号理财计划产品说明

## 所有文档中是否都正常提取出4个表

In [13]:
for name,value in clean_tables.items():
    if len(value) == 2:
        print(name)
    print(len(value))

3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
招商银行零售青葵系列一年半定开4号理财计划产品说明书（产品代码：301040）.xml
2
招商银行零售青葵系列一年半定开5号理财计划产品说明书.xml
2
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
6
4
4
4
4
4
4
4


## 观察其中一个

In [15]:
for table in clean_tables['招商银行零售青葵系列一年半定开5号理财计划产品说明书.xml']:
    table

,0,1
0,名称,招商银行零售青葵系列一年半定开 5号理财计划（产品代码： 301041）
1,全国银行业理财信息登记系统登记编号,产品登记编码：C1030819000993，投资者可以根据该登记编号在中国理财网（网址： h...
2,理财币种,人民币
3,产品类型,固定收益类
4,募集方式,公募发行
5,运作方式,定期开放式
6,业绩比较基准,A份额：首个运作期年化 3.80%-4.20%，A份额首个运作期自 2019年 7月 30日...
7,本金及理财收益（如有，下同）,投资者持有本理财计划到期，本理财计划不保障本金且不保证理财收益。在理财计划到期前，本理财计划...
8,理财计划份额,理财计划份额以人民币计价，单位 1份。
9,理财计划份额面值,每份理财计划份额面值为人民币 1元。


,0,1
0,投资品种,计划配置比例（占净资产的比重）
1,货币市场和现金管理工具、标准化债权资产,不低于 60%
2,其中：高流动性资产,不低于 5%
3,非标准化债权资产,0%-40%
4,衍生金融工具（以保证金计）,0%-5%
